In [2]:
# Importing recquired packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [3]:
# Locating the driver
driver_location = "/Users/camilleepitalon/chromedriver"
driver = webdriver.Chrome(driver_location)

# Activating the driver on frimm landing page
driver.get('https://www.frimm.com/index.php?ac=frimm_cerca_agenzia_immobiliare')

In [4]:
# Creating the list of cities 
# No result for Genova, Catania, Firenze and Bari
cities_list = ['Roma', 'Milano', 'Napoli', 'Torino', 'Palermo','Bologna, BO']

In [5]:
df = pd.DataFrame(columns=['Title','Agency_ID','Street','Zip_Code','City','Zone','Phone','Email','Website'])

for city in cities_list:

    # Starting research
    driver.find_element_by_xpath("//input[@class='search__input pac-target-input']").clear()
    driver.find_element_by_xpath("//input[@class='search__input pac-target-input']").send_keys(city,Keys.ENTER)
    time.sleep(3)

    
    # Getting the number of results pages
    number_of_pages_text = driver.find_element_by_xpath("//span[@id='totale_risultati_1']").text
    number_of_pages = int(number_of_pages_text[-1])
    
    # Getting all results of the research
    for i in range(number_of_pages-1):
        driver.find_element_by_xpath("//a[text()='Mostra ulteriori risultati']").click()
        time.sleep(2)
    
    # Extracting all boxes
    boxes_list = driver.find_elements_by_xpath('//div[@class="media-body"]')

    for box in boxes_list:
        text = box.text
        text_splited = text.split('\n')

        # Agency name and ID
        title = text_splited[0]
        agency_id = int(text_splited[1].replace('ID:',''))

        #Address elements
        address_list = text_splited[2].split(' | ')
        street = address_list[0]
        if len(address_list) == 4:
            zip_code = address_list[1]
            city = address_list[2]
            zone = address_list[3][6:]
        elif len(address_list) == 2:
            zip_code = None
            city = address_list[1]
            zone = None

        # Agency contact elements
        contact_list = text_splited[3].split('  ')
        if len(contact_list) == 3:
            phone = contact_list[0][1:]
            email = contact_list[1]
            website = contact_list[2]
        elif len(contact_list) == 2:
            phone = contact_list[0][1:]
            if '@' in contact_list[1]:
                email = contact_list[1]
                website = None
            elif 'www' in contact_list[1]:
                email = None
                website = contact_list[1]
            else :
                email = None
                website = None

        box_dict = {'Title':title,'Agency_ID':agency_id,'Street':street,'Zip_Code':zip_code,'City':city,'Zone':zone,'Phone':phone,'Email':email,'Website':website}
        df = df.append(box_dict, ignore_index=True)
        
print(df)

                                                Title Agency_ID  \
0                         AFFILIATO FRIMM TIBURTINA           2   
1                       Solari Real Estate S.R.L.S.          39   
2                    Casagregoryre Srl Semplificata          66   
3            Selling Immobiliare Di Paola Peppoloni         190   
4                Affiliato Frimm Mambor Immobiliare         218   
5                        EffettoCasa Srl Aff. Frimm         228   
6                             FRALE IMMOBILIARE SNC         235   
7            ElEman S.R.L. Di Eleuterio Ed Emanuele         243   
8                        Immobiliare Salaria S.R.L.         293   
9                       Gruppo Immobiliare 2000 Sas         301   
10                    Immobiliare Napoli Centro Sas         248   
11                                     Frimm Vomero         569   
12                          Frimm Affiliato Pianura        2912   
13             Academy Napoli S.R.L Roberto Barbato       4291

In [51]:
df.to_csv(r'/Users/camilleepitalon/Python/Scraping_frimm/output_2.csv', index = False)